# 回顾
1. mvc ---> mvt
2. virtualenv --no-site-packages 版本名
3. 进入虚拟环境
   - windows:
      - cd Scripts 执行activate 退出deactivate
   - linux:
      - source bin/activate 退出 source deactivate
4. 安装django
    pip install django==1.11
5. 创建django工程
    django-admin startproject helloworld
6. 启动项目
   - python manage.py runserver IP:端口
7. 基础文件
   - __init__.py: 初始化，配置pymysql；链接的地方
   - settings.py: 配置信息位置，databases等
   - urls.py: url路由
   - wsgi.py:网关
8. 创建app
   - python manage.py startapp app_name

9. 配置
   1. settings.py文件中INSTALLED_APPS中写入创建的app的name
   2. 也可以导入app.py文件中的APPCONFIG下的name属性
10. 模型
    - 在models.在models.py文件中定义class 模型名称继承models.Model
    - db_tables:定义数据库中的表名称
11. 迁移
    - python manage.py makemigrations
    - python manage.py migrate
12. 保存数据
    - stu = Student()
    - stu.name = 'xxx'
    - stu.save()



## 设置超级管理员登录

- 创建超级管理员账号和密码
    - python manage.py createsuperuser 

### 设置管理员界面
- 在app下的admin.py文件中

In [ ]:
# -*- coding:utf-8 -*-
from django.contrib import admin

# Register your models here.
from stu.models import Student


# 2.第二种注册方式
@admin.register(Student)
class StudentAdmin(admin.ModelAdmin):

    def set_sex(self):
        if self.sex:
            return '男'
        else:
            return '女'
    # 修改性别字段的描述
    set_sex.short_description = '性别'
    # 展示字段
    list_display = ['id', 'name', set_sex]
    # 过滤
    list_filter = ['name']
    # 搜索
    search_fields = ['name']
    # 分页
    list_per_page = 2


# 1. 注册的第一种方式，这种方式不用加装饰器
# admin.site.register(Student, StudentAdmin)

- ORM（objects relational mapping）
- 对象关系映射，翻译机
- 模型字段
   - CharField: 字符串
      - max_length:长度
   - BooleanField: 布尔类型
      - 1 0
   - DateField: 年月日，日期
      - auto_now_add=True 自动生成第一创建时的时间
      - auto_now=True ，每次修改是时候赋值
   - DateTimeField:年月日
      - auto_now_add=True 自动生成第一创建时的时间
      - auto_now=True ，每次修改是时候赋值
   - AutoField: 自动增长
   - DecimalField(max_digits=a, decimal_places=b)
      - max_digits 指定这个数字最大有多少位
      - decimal_places 小数有多少位
   - TextField: 存文本信息，适合存很大的文本
   - IntergerField: 整数
   - FloatField: 浮点数
   - FileField: 文件上传字段
   - ImageField:上传图片
      - upload_to='' 指定上传图片路径
- 模型参数
   - default :默认
   - null :设置是否为空，针对数据库该字段是否为空
   - blank : 页面可以bu， 针对表单提交该字段是否为空
   - primarykey :创建主键
   - unique : 唯一

1. 修改字段名
   - alter table table_name change 原名称 修改后名称 类型

2. objects对象
   - 通过模型.objects来实现数据的CRUD操作

3. 获取所有学生
   - select * from student;
   - 模型.objects.all()

4. 获取所有女生的信息
   - filter(过滤条件)
   - get(过滤条件)
   - 区别：
   - get：返回一个满足条件的对象，没有满足条件的则直接报DoesNotExit的异常，
   - 如果查询结果有多个数据的话，就报MulitiObjectsReturned
   - filter（）：返回满足条件的结果 
   - first（）：返回第一条数据
   - last（）：返回最后一条数据
   - count（）：求和
   - gt gte：大于  大于等于
   - lt lte： 小于  小于等于
5. F() / Q()
   - from django.db.models import F，Q

## 应用

- 在主项目目录下的__init__.py文件中

In [ ]:
import pymysql

pymysql.install_as_MySQLdb()

- 在app目录下的models.py文件中

In [ ]:
from django.db import models

# Create your models here.


class Student(models.Model):
    #下面这段话不需要写，系统会自动生成
    #stu_id = models.AutoField(primary_key=True)
    stu_name = models.CharField(max_length=6)
    stu_sex = models.BooleanField(default=0)
    stu_birth = models.DateField()
    stu_delete = models.BooleanField(default=0)
    stu_create_time = models.DateField(auto_now_add=True)
    stu_operate_time = models.DateField(auto_now=True)
    stu_tel = models.CharField(max_length=11)
    stu_yuwen = models.DecimalField(max_digits=3, decimal_places=1, default=0)
    stu_math = models.DecimalField(max_digits=3, decimal_places=1, default=0)

    class Meta:
        db_table = 'stu'


- 在app下的views.py文件中操作下面代码

In [ ]:
from django.db.models import F, Q
from django.http import HttpResponse
from django.shortcuts import render

# Create your views here.
from stu.models import Student


def hello(request):
    return HttpResponse('hello')


def addStu(request):
    # 添加学生信息
    if request.method == 'GET':
        return render(request, 'index.html')
    if request.method == 'POST':
        # 处理提交的学生信息
        stu_name = request.POST.get('name')
        if request.POST.get('sex') == '男':
            stu_sex = 1
        else:
            stu_sex = 0
        stu_birth = request.POST.get('birth')
        stu_tel = request.POST.get('tel')

        # stu = Student()
        # stu.stu_name = stu_name
        # stu.stu_sex = stu_sex
        # stu.stu_birth = stu_birth
        # stu.stu_tel = stu_tel
        # stu.save()
        # 这两种方法都是想数据库添加数据，但明显下面的方法更加简单
        Student.objects.create(
            stu_name=stu_name,
            stu_sex=stu_sex,
            stu_birth=stu_birth,
            stu_tel=stu_tel
        )

        return HttpResponse('添加学生信息成功')


def selectStu(request):
    # 查询
    # stus = Student.objects.all()
    # stus = Student.objects.filter(stu_sex=False)
    # stus = Student.objects.get(stu_sex=False)
    # 查询id从大到小的排序
    # stus = Student.objects.all().order_by('-id')
    # 获取id最大的数据
    # stus = Student.objects.all().order_by('-id').first()
    # 获取id最小的数据
    #stus = Student.objects.all().order_by('-id').last()
    # 获取男生的数据的个数
    # stus_count = Student.objects.filter(stu_sex=True).count()
    # print(stus_count)
    # 查询所有80后女生的信息
    # stus = Student.objects.filter(stu_sex=False).\
    #     filter(stu_birth__gte='1980-01-01').\
    #     filter(stu_birth__lt='1990-01-01')
    # stus = Student.objects.filter(stu_sex=False,
    #                               stu_birth__gte='1980-01-01',
    #                               stu_birth__lt='1990-01-01')
    # 查询姓李的数据
    # stus = Student.objects.filter(stu_name__startswith='李')
    # 查询姓名以白为结束的数据
    # stus = Student.objects.filter(stu_name__endswith='助')
    # 姓名中包含李的数据
    # stus = Student.objects.filter(stu_name__contains='李')
    # 判断是否存在张三
    # is_stu = Student.objects.filter(stu_name='张三').exists()
    # print(is_stu)
    #获取制定多个id的值
    # ids = [1, 2, 3]
    # stus = Student.objects.filter(id__in=ids)
    # 查询语文成绩大于数据成绩的学生
    # stus = Student.objects.filter(stu_yuwen__gte=F('stu_math'))
    # 查询语文成绩超过数学成绩十分的学生
    # stus = Student.objects.filter(stu_yuwen__gte=F('stu_math') + 10)
    # 查询学生姓名不叫李白的，或者语文成绩大于80分的学生
    # ~ 不叫李白
    # stus = Student.objects.filter(~Q(stu_name='李白') | Q(stu_yuwen__gt=80))
    # 查询学生姓名不叫李白的，并且语文成绩大于80分的学生
    stus = Student.objects.filter(~Q(stu_name='李白') & Q(stu_yuwen__gt=80))
    # 返回给前端
    return render(request, 'sel_stu.html', {'stus': stus})

- 在主项目目录下创建一个名为templates的文件夹
- 修改setting文件中下面内容中的DIRS

In [ ]:
TEMPLATES = [
    {
        'BACKEND': 'django.template.backends.django.DjangoTemplates',
         # new 增加了路径，以便可以找到templates路径下的内容
        'DIRS': [os.path.join(BASE_DIR, 'templates')], 
        'APP_DIRS': True,
        'OPTIONS': {
            'context_processors': [
                'django.template.context_processors.debug',
                'django.template.context_processors.request',
                'django.contrib.auth.context_processors.auth',
                'django.contrib.messages.context_processors.messages',
            ],
        },
    },
]

- 注释掉setting文件下的一段内容，以便可以通过post,和get请求

In [ ]:
MIDDLEWARE = [
    'django.middleware.security.SecurityMiddleware',
    'django.contrib.sessions.middleware.SessionMiddleware',
    'django.middleware.common.CommonMiddleware',
    # 'django.middleware.csrf.CsrfViewMiddleware', # 注释这段是为了可以通过post请求
    'django.contrib.auth.middleware.AuthenticationMiddleware',
    'django.contrib.messages.middleware.MessageMiddleware',
    'django.middleware.clickjacking.XFrameOptionsMiddleware',
]

- index.html
- 实现在页面给数据库传输数据
```HTML
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>Title</title>
</head>
<body>
    <form action="/stuapp/addstu/" method="post">
    <table>
        <tr>
            <td>姓名</td>
            <td>性别</td>
            <td>生日</td>
            <td>电话</td>
        </tr>
        <tr>
            <td><input type="text" name="name"></td>
            <td><input type="text" name="sex"></td>
            <td><input type="date" name="birth"></td>
            <td><input type="text" name="tel"></td>
        </tr>
    </table>
    <input type="submit" value="提交">
    </form>
</body>


```

- sel_stu.html
- 接收可迭代的对象
```HTML
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>查询所有学生信息</title>
</head>
<body>
    <!-- {% %}中写条件语句 -->
    {% for stu in stus %}
    <!-- {{}}中可以接收变量 -->
        id : {{ stu.id }}
        姓名：{{ stu.stu_name }}
        生日：{{ stu.stu_birth }}
        电话：{{ stu.stu_tel }}
        <br />
    {% endfor %}
</body>
</html>
```

- detail.html
- 用来接收不可迭代的变量
```HTML
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>Title</title>
</head>
<body>
id:{{ stus.id }}
name:{{ stus.stu_name }}
</body>
</html>
```

In [ ]:
"""
班级表 学生表
1.url --> 班级页面， 展示班级信息,
2.所有学生信息页面
"""